In [9]:
# Liturgický kalendár ako "one page".
# zobrazenie pre web (použitá mustra_web.html)
# testované na zip archíve https://lc.kbs.sk/files/LC2024.zip

In [10]:
import pandas as pd
from bs4 import BeautifulSoup, Comment
import pickle
import re

In [11]:
# Load the dictionary from the pickle file
with open('dict_of_strings.pickle', 'rb') as file:
    dict_of_strings = pickle.load(file)

In [12]:
with open('mustra_web.html', mode='r', encoding='utf-8') as m:
    mustra = m.read()
    
soup1 = BeautifulSoup(mustra,'lxml')

my_table = soup1.new_tag('table') 
soup1.body.append(my_table)

In [13]:
for i, (date_string, file_content) in enumerate(dict_of_strings.items()):

    # týždne sú oddelené hrubšími čiarami, jednotlivé týždne sú zabalené v <tbody> elementoch
    current_date = pd.to_datetime(date_string)
    if (i == 0) or current_date.day_name('sk_SK')=='Pondelok':
        tbody_here = soup1.new_tag('tbody') # create new tbody here
        soup1.table.append(tbody_here)

    new_table_row = soup1.new_tag('tr')
    tbody_here.append(new_table_row)

    ### soup2
    soup2 = BeautifulSoup(file_content,'lxml')
    
    # h1
    lcPREV = soup2.h1.find('a', class_='lcPREV')
    if lcPREV:
        lcPREV.extract()

    lcNEXT = soup2.h1.find('a', class_='lcNEXT')
    if lcNEXT:
        lcNEXT.extract()
        
    # mažeme link, aj nadbytočný <span> :
    soup2.find('a', class_='lcMY').unwrap() # alebo: soup2.h1.a.unwrap() #  lcMY
    soup2.find('span', class_='lcMD').unwrap() # lcMD
    span_lcDMY = soup2.find('span', class_='lcDMY') # lcDMY
    span_lcDMY.smooth()
    span_lcDMY.string = span_lcDMY.string[:-5] # mažeme rok 2023 z textu 
    # toto nešlo: span_lcDMY.string = span_lcDMY.string.removesuffix(' 2023') # 

    del soup2.h1['nav'] # removing nav attribute
    del soup2.h1['class'] # removing class attribute
    # soup2.smooth() # nevyžehlí to... hmm...

    link_string=date_string+'.html'
    td1 = soup1.new_tag('td', id=date_string, onclick=f"mf(this.id)") # id='2023-01-01' napríklad. využije sa pri naDnes navigácii
    now_here = td1 # onclick vetva

    now_here.append(soup2.h1) # vkladáme hlavičku (dátum)

    for elem in soup2.find_all('div', class_='lcHEAD'): # vkladáme lcHEAD (môže ich byť viac)

        text = elem.h2.get_text()
        if 'Fakultatívne' in text: # ak je to Fakultatívne čítanie, mažeme.
            # print("fak")
            continue
            # elem.extract() 

        h3 = elem.find('h3')
        if h3 and h3.get_text().startswith('Vigília'): # ak ide o vigíliu, nezapisujeme do tabulky 
            continue # 

        nestandardna_spomienka = elem.find('p', class_='lcTEXT') # ak chcem dávať aj Extra spomienky v Advente / Pôste...

        elem.find('div', class_='lcHEADinfo').extract() # mažeme súradnice
        del elem.h2['title'] # removing hover_over_title
        del elem.h2['id'] # removing id="toc_"
        
        for e in elem.find_all('span', class_='lcFARBA'):
            del e['title']
            e.string = '' # "\xa0" # https://stackoverflow.com/questions/22187049/how-to-insert-a-blank-spacenbsp-into-a-beautifulsoup-tag
            par = e.find_parent() # premiestňujem za span (lebo farba je vnorená v parent spane.)
            par.insert_after(e)

        ### TODO: kašlať na to, spravím to cez regex nakoniec.
        # for e in elem.find_all('h2'):
        #     # print(e.span)
        #     if e.span.string:
        #         # print(e.span.string)
        #         e.span.string = e.span.string.strip(' \n')
        #         # print(e.span.string)
        #     else: 
        #         print(e) # nechápem.
        #
        #   # e.span.string = e.span.string[:-1]
        #   # e.span.contents.pop()
        #   # tu musím zmazať vo všetkých h2.span medzery na konci
        #   # elem.h2.span.contents.pop() # neviem prečo je na konci ' ', mažem.

        for e in elem.find_all('span', class_='lcMOD', string='alebo'): # musí byť string 'alebo', lebo môže byť aj 'večer'
            e.string='al.'
            # e.extract() # mažem 'alebo'
        
        for e in elem.find_all('i', class_="lcTYP", string='(ľubovoľná spomienka)'):
            e.extract() # mažem popisok '(ľubovoľná spomienka)'

        for e in elem.find_all('i', class_="lcTYP"): # premiestňujem lcTYP pred farbu.
            farba = e.find_previous_sibling() # .span # find('span', class_='lcFARBA')
            e.insert_after(farba) # farba na koniec! 

        lcGEN = elem.find('div', class_='lcGEN') # "Dáta nie sú skontrolované podľa direktória."
        if lcGEN:
            # lcGEN_text = lcGEN.get_text(strip=True)
            elem.h2.contents.pop() # neviem prečo je na konci '\n', mažem.
            del lcGEN['title'] # removing title attribute
            elem.h2.append(lcGEN)
            # lcGEN.extract()
            # print(elem.h2.contents)
            # farba_znovu = elem.find_all('span', class_='lcFARBA')
            # farba_znovu.insert_ater(lcGEN.get_text(strip=True))

        if nestandardna_spomienka:
            my_text:str = nestandardna_spomienka.getText()
            if my_text.startswith('Spomienka'):
                my_text = my_text.replace('Spomienka ','(')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z&nbsp;jeho spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z&nbsp;jej spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z&nbsp;ich spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z jeho spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z jej spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z ich spomienky.',')')  
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z jeho spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z jej spomienky.',')')
                my_text = my_text.replace(', môže byť tak, že sa namiesto kolekty dňa berie kolekta z ich spomienky.',')')                               
                special = BeautifulSoup(f'<br><span class="sp">{my_text}</span>','html.parser')
                elem.h2.append(special)
                # now_here.append(my_text) # musím vložiť ešte do h2 !    
                    
        now_here.append(elem)        
    
    new_table_row.append(td1) # <tr><td>    

In [14]:
# final cleaning (size reducing)

# removing html comments <!-- -->
comments = soup1.find_all(string = lambda x: isinstance(x, Comment))
for comment in comments:
    comment.extract()

# soup1.smooth() # optionally

# with open('1p4_app_onclick2024_23.html', mode='w', encoding='utf-8') as f_out:
#     f_out.write(str(soup1))

In [15]:
pat = r'<span>\n(.+)\n</span>'
new = r'<span>\1</span>'
s=str(soup1)
new_s=re.sub(pat,new,s)

# new_s=new_s.replace(' ',' ') # replace nbsp with normal space.

In [16]:
with open('2024.html', mode='w', encoding='utf-8') as f_out:
    f_out.write(new_s)

In [ ]:
# done.